In [1]:
import numpy as np
import math

## Input data

In [2]:


f = open("input.txt", "r")
data = list(filter(lambda arr: len(arr) > 0,
                   list(map(lambda line: list(map(lambda y: int(y),
                                               filter(None ,line.split(' ') ))),
                            f.read().split('\n')))))
alternatives = data[0:len(data)-1]
weights = data[len(data)-1:len(data)][0]
f.close()

print("Alternatives\n{}".format(np.array(alternatives)))
print("Weights\n{}".format(np.array(weights)))

Alternatives
[[ 3  1  1 10  6  7 10  8  6  6  1  1]
 [ 7  5  7  6  2  6  7  6 10  7  7  9]
 [ 3  4  1  1  7  8  6  5  9  2  6  3]
 [10  5  8  2  9  9  6  4  4 10  3  8]
 [ 2  2  9  4  1  4  2  4  6  4  5  5]
 [ 1  6 10  7  5  8  9  3  6  3  5  3]
 [ 1  6  4  9  3  8  6 10 10  5  6  2]
 [ 9  7 10  8  9  5  9  8  4  8  3  1]
 [ 9  2  4  2  5  3  4  2  8  5  7  4]
 [ 3  1  3 10  9  6  3  4  3  9  9  9]
 [ 7  2  4  9  9  8 10  6  4  1  1  5]
 [ 1  2  9  5  8  3  5  9  2  3  8  6]
 [ 5  5  7  9  9 10  7  1 10  6  3  1]
 [ 1  5  6  2  1  1  3  8  2  4  2  6]
 [ 9  2  9  3  3  4  7 10  8  4 10  1]]
Weights
[ 8 10  9 10  1  3  7  9  4  1 10  2]


## Utility functions

In [3]:
# rationing of weights
def normalize_weights(weight_arr):
    sum_weights = sum(weight_arr)
    normalized_weights = []
    for i in range(0,len(weight_arr)):
        normalized_weights.append(weight_arr[i]/sum_weights)
    return normalized_weights

# multiplication of the matrix by the matrix of criteria weights
def weighted_estimates(r_matrix, weight_arr):
    norm_weights = normalize_weights(weight_arr)
    for i in range(0, 15):
        for j in range(0, 12):
            r_matrix[i][j] = r_matrix[i][j] * norm_weights[j]
    return r_matrix

# output results
def output_result(result, opt, method):
    print(method)
    print("Ranking:")
    for i in result:
        print(i, end=" ")
    print()
    if len(opt)==1:
        print("Best alternative:")
    else:
        print("Array of the best alternatives:")
    for i in opt:
        print(i, end=" ")
    print()

## TOPSIS

In [4]:
# normalization of parameters for the case of maximization of all criteria
def normalize_estimates_uniform(_alternatives, weights_count):
    r_matrix = [[0]*weights_count for _ in range(len(_alternatives))]
    a = np.array(_alternatives)
    pows = []
    for k in range(0,weights_count):
        column = a[:,k]
        for i in column:
            i = pow(i, 2)
        pows.append(sum(column)) 
    for i in range(0, len(_alternatives)):
        for j in range(0, weights_count):
            r_matrix[i][j] = _alternatives[i][j]/(math.sqrt(pows[j]))
    return r_matrix

# normalization at k1-k7 are subject to maximization, and criteria k8-k12 - are minimized
def normalize_estimates_kplus_kminus(alt):
    alternative_arr = np.array(alt)
    kplus_criteria = np.empty((15,7))
    kminus_criteria = np.empty((15,5))
    r_matrix = np.empty((15,12))
    for i in range (0,15):
        for j in range (0,7):
            kplus_criteria[i][j] = alternative_arr[i][j]
    for i in range (0,15):
        for j in range (7,12):
            kminus_criteria[i][j-7] = alternative_arr[i][j]
    # normalization of the criteria to be maximized
    for i in range(0, 15):
        for j in range(0, 7):
            min_kplus = min(kplus_criteria[:,j])
            max_kplus = max(kplus_criteria[:,j])
            kplus_criteria[i][j] = (kplus_criteria[i][j]-min_kplus)/(max_kplus-min_kplus)
    # normalization of criteria to be minimized
    for i in range(0, 15):
        for j in range(0, 5):
            min_kminus = max(kminus_criteria[:,j])
            max_kminus = min(kminus_criteria[:,j])
            kminus_criteria[i][j] = (min_kminus-kminus_criteria[i][j])/(min_kminus-max_kminus)
    for i in range (0,15):
        for j in range (0,7):
            r_matrix[i][j] = kplus_criteria[i][j]
    for i in range (0,15):
        for j in range (7,12):
            r_matrix[i][j] = kminus_criteria[i][j-7]
    return r_matrix

# calculation of distances to utopian and anti-utopian point
def calculate_d_pis_nis(row_i, max_j, min_j):
    r_pis = []
    r_nis = []
    for i in range(0, len(row_i)):
        r_pis.append(pow((row_i[i]-max_j[i]), 2))
        r_nis.append(pow((row_i[i]-min_j[i]), 2))
    d_pis = math.sqrt(sum(r_pis))
    d_nis = math.sqrt(sum(r_nis))
    return d_pis, d_nis

# Establish the proximity of each alternative to a positive ideal point
def calculate_c(d_pis, d_nis):
    c = []
    for i in range(0, len(d_pis)):
        c.append(d_nis[i]/(d_pis[i] + d_nis[i]))
    return c

# TOPSIS
def topsis(alternative_arr, weight_arr, task):
    if task == "a":
        # normalization by the general formula
        norm_a = normalize_estimates_uniform(alternative_arr, len(weight_arr))
    else:
        # normalization according to formulas for profit criteria and cost criteria
        norm_a = normalize_estimates_kplus_kminus(alternative_arr)
    # Calculation of weighted normalized estimates of alternatives
    weighted_a = np.array(weighted_estimates(norm_a, weight_arr))
    max_j = []
    min_j = []
    for j in range(0, 12):
        # definition of utopian and anti-utopian point
        max_j.append(max(weighted_a[:,j]))
        min_j.append(min(weighted_a[:,j]))
    d_pis = []
    d_nis = []
    for i in range(0, 15):
        # distances to utopian and anti-utopian points
        d_p, d_n = calculate_d_pis_nis(weighted_a[i], max_j, min_j)
        d_pis.append(d_p)
        d_nis.append(d_n)
    # calculation of proximity to utopian and anti-utopian point
    c = calculate_c(d_pis, d_nis)
    indexes = np.argsort(c)
    for i in range(0,len(indexes)):
        indexes[i]+=1
    result = indexes[::-1]
    opt = [result[0]]
    return result, opt

def print_topsis():
    result_a, opt_a = topsis(alternatives, weights, "a")
    result_b, opt_b = topsis(alternatives, weights, "b")
    output_result(result_a, opt_a, "TOPSIS - all criteria need to be maximized")
    print("------------------------------------------------------------")
    output_result(result_b, opt_b, "TOPSIS - k1-k7 are subject to maximization, and k8-k12 are minimized")

print_topsis()

TOPSIS - all criteria need to be maximized
Ranking:
8 2 15 7 6 12 13 4 10 11 1 9 5 14 3 
Best alternative:
8 
------------------------------------------------------------
TOPSIS - k1-k7 are subject to maximization, and k8-k12 are minimized
Ranking:
13 8 11 6 4 15 14 2 7 1 9 5 12 10 3 
Best alternative:
13 


## VIKOR

In [5]:
# formation of sets of desired and worst values
def calculate_f_max_min_values(alt):
    alternative_arr = np.array(alt)
    max_f = []
    min_f = []
    for j in range (0,12):
        max_f.append(max(alternative_arr[:,j]))
        min_f.append(min(alternative_arr[:,j]))
    return max_f, min_f

# normalization of criteria
def create_vikor_matrix(alternative_arr, max_f, min_f):
    vikor_matrix = np.empty((15,12))
    for i in range(0,15):
        for j in range(0,12):
            vikor_matrix[i][j] = (max_f[j]-alternative_arr[i][j]) / (max_f[j]-min_f[j])
    return vikor_matrix

# calculate the average interval for improving the alternative
def calculate_sk(weighted_vikor_matrix):
    sk = []
    for i in weighted_vikor_matrix:
        sk.append(sum(i))
    max_sk = max(sk)
    min_sk = min(sk)
    return sk, max_sk, min_sk

# calculation of the maximum interval of alternative improvement
def calculate_rk(weighted_vikor_matrix):
    rk = []
    for i in weighted_vikor_matrix:
        rk.append(max(i))
    max_rk = max(rk)
    min_rk = min(rk)
    return rk, max_rk, min_rk

# verification of compliance with conditions C1 and C2
def check_c1_c2(q, min_sk, min_rk, q_i, sk_i, rk_i):
    check_c1 = False
    check_c2 = False
    if q[1]-q[0]>=1/14:
        check_c1 = True
    if q_i[0] == sk_i[0] or q_i[0] == rk_i[0]:
        check_c2 = True
    return check_c1, check_c2

def vikor(alternative_arr, v):
    # sets of desired and worst values
    max_f, min_f = calculate_f_max_min_values(alternative_arr)
    # normalization and consideration of weight criteria
    vikor_matrix = create_vikor_matrix(alternative_arr, max_f, min_f)
    weighted_vikor_matrix = weighted_estimates(vikor_matrix, weights)
    # average intervals for improving alternatives
    sk, max_sk, min_sk = calculate_sk(weighted_vikor_matrix)
    # maximum intervals for improving alternatives
    rk, max_rk, min_rk = calculate_rk(weighted_vikor_matrix)
    q = []
    for i in range(0,15):
        # Calculate the values of Qk, k = 1,2 ..., n for each alternative
        q.append(v*(sk[i]-min_sk)/(max_sk-min_sk)+(1-v)*(rk[i]-min_rk)/(max_rk-min_rk))
    q_indexes = np.argsort(q)
    sk_indexes = np.argsort(sk)
    rk_indexes = np.argsort(rk)    
    print("Ranking Q:")
    for i in q_indexes:
        print(i+1, end=" ")
    print()
    print("Ranking S")
    for i in sk_indexes:
        print(i+1, end=" ")
    print()
    print("Ranking R")
    for i in rk_indexes:
        print(i+1, end=" ")
    print()
    q_sorted = sorted(q)
    # verification of compliance with conditions C1 and C2
    check_c1, check_c2 = check_c1_c2(q, min_sk, min_rk, q_indexes, sk_indexes, rk_indexes)
    opt = []
    opt_values = []
    #output, if the conditions are met
    if check_c1==True and check_c2==True:
        opt.append(q_indexes[0]+1)
        opt_values.append(q[0])
    # create a set of better alternatives if C1 does not hold
    elif check_c1==False and check_c2==True:
        print ("C1 are not met")
        print()
        opt.append(q_indexes[0]+1)
        opt_values.append(q[0])
        for i in range(1,len(q)):           
            if q[i]-q[i-1] < 1/14:
                opt.append(q_indexes[i]+1)
                opt_values.append(q[i])
            else:
                break
    # creating many better alternatives if C2 is not met
    elif check_c1==True and check_c2==False:
        print ("C2 are not met")
        print()
        opt.append(q_indexes[0]+1)
        opt_values.append(q[0])
        opt.append(q_indexes[1]+1)
        opt_values.append(q[1])
    else:
        print ("С1 and С2 are not met")
        print()
    for i in range(0,len(q_indexes)):
        q_indexes[i]+=1
    return q_indexes, opt, opt_values

def print_vikor():
    result, opt, opt_values = vikor(alternatives, 0.5)
    output_result(result, opt, "VIKOR - v=0.5")
    print("Q values of best alternatives:")
    for i in opt_values:
        print(i, end=' ')
    print()
    print("RESEARCH")
    print("______________________________________________________________")
    v_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    for v in v_values:
        result, opt, opt_values = vikor(alternatives, v)
        output_result(result, opt, "VIKOR - v={}".format(v))
        print("Q values of best alternatives:")
        for i in opt_values:
            print(i, end=' ')
        print()
        print("______________________________________________________________")
        
print_vikor()

Ranking Q:
2 8 7 15 6 13 4 12 11 5 9 10 1 14 3 
Ranking S
8 2 15 7 6 13 4 11 12 10 1 9 3 5 14 
Ranking R
2 8 6 7 5 12 15 4 9 14 13 1 3 10 11 
C1 are not met

VIKOR - v=0.5
Ranking:
2 8 7 15 6 13 4 12 11 5 9 10 1 14 3 
Array of the best alternatives:
2 8 
Q values of best alternatives:
0.8731481481481479 0.08819444444444445 
RESEARCH
______________________________________________________________
Ranking Q:
2 8 6 7 5 12 15 4 9 14 13 1 3 10 11 
Ranking S
8 2 15 7 6 13 4 11 12 10 1 9 3 5 14 
Ranking R
2 8 6 7 5 12 15 4 9 14 13 1 3 10 11 
C1 are not met

VIKOR - v=0.0
Ranking:
2 8 6 7 5 12 15 4 9 14 13 1 3 10 11 
Array of the best alternatives:
2 8 
Q values of best alternatives:
1.0 0.0 
______________________________________________________________
Ranking Q:
2 8 7 6 15 12 5 4 13 9 14 11 10 1 3 
Ranking S
8 2 15 7 6 13 4 11 12 10 1 9 3 5 14 
Ranking R
2 8 6 7 5 12 15 4 9 14 13 1 3 10 11 
C1 are not met

VIKOR - v=0.1
Ranking:
2 8 7 6 15 12 5 4 13 9 14 11 10 1 3 
Array of the best alternat